In [1]:
import numpy as np 
import gym  
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
import pickle

### Prepare data

In [2]:
data_path="expert_data/Pendulum-v1_10_-130.pkl"

In [3]:
with open(data_path, "rb") as f:
    data_good = pickle.load(f)
print('expert data loaded')

data_good=data_good[:20]

good_obs=[]
good_acts=[] 
for traj in data_good: 
    s,a,r=traj   
    good_obs.append(s)
    good_acts.append(a) 

states=np.vstack(good_obs)
actions=np.vstack(good_acts)
print('X:',states.shape,' y:', actions.shape)

expert data loaded
X: (2000, 3)  y: (2000, 1)


In [4]:
action_dim=actions.shape[1]
state_dim=states.shape[1]
print(state_dim, action_dim)

3 1


### Train Model

In [5]:
model=LinearRegression()
model.fit(states, actions)
score=model.score(states, actions)
print('score:', score)
#Not very good, so we will use randomforestregressor instead

score: 0.2268964998032269


In [6]:
model=RandomForestRegressor()
model.fit(states, actions)
score=model.score(states, actions)
print('score:', score)
#Better, we will use this model as bc.

/home/ns/anaconda3/lib/python3.9/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


score: 0.9817352364111102


### Inference

In [7]:
env_name='Pendulum-v1'
env = gym.make(env_name)
env.action_space

Box(-2.0, 2.0, (1,), float32)

In [8]:
def play_an_episode(env_name, model, render=False, max_step=500):
    if render:
        env=gym.make(env_name, render_mode='human')
    else:
        env = gym.make(env_name)
    total_reward=0  
    state,info=env.reset() 
    for i in range(max_step):
        if render: env.render()
        action = model.predict([state])[0] 
        next_state,reward,done,trunc,info=env.step([action])
        total_reward+=reward
        state=next_state
        if done or trunc: 
            break 
    
    env.close()
    return total_reward

In [11]:
r=play_an_episode(env_name, model, render=True)
print('reward:', r)

reward: -483.61055463535604


In [9]:
scores=[]
n_trajectory=20
for i in range(n_trajectory):
    rewards=play_an_episode(env_name, model)
    print(f'episode #{i} reward: {rewards:0.2f}')
    scores.append(rewards)

print(f'mean score: {np.mean(scores):0.2f} std: {np.std(scores):0.2f}')

episode #0 reward: -0.99
episode #1 reward: -353.93
episode #2 reward: -1049.09
episode #3 reward: -745.35
episode #4 reward: -120.54
episode #5 reward: -897.87
episode #6 reward: -1.24
episode #7 reward: -882.04
episode #8 reward: -1045.98
episode #9 reward: -894.47
episode #10 reward: -246.74
episode #11 reward: -770.96
episode #12 reward: -889.65
episode #13 reward: -0.48
episode #14 reward: -1063.61
episode #15 reward: -1047.45
episode #16 reward: -1112.95
episode #17 reward: -1052.66
episode #18 reward: -874.75
episode #19 reward: -1047.75
mean score: -704.93 std: 401.29
